### 국가별 백신 접종률 정보 가져오기

### 정보 수집 사이트 
* bloomberg.com
  * https://www.bloomberg.com/graphics/covid-vaccine-tracker-global-distribution/

### 기타 참고 통계 사이트
* our world in data
  * https://ourworldindata.org/covid-vaccinations


In [1]:
from IPython.display import display, Image
import os, warnings
import re
warnings.filterwarnings(action='ignore')

### 01 웹 브라우저 띄우기
 *  만약 chrome 브라우저와 chromedirver의 버전이 안 맞을 경우, 버전을 맞는 것으로 변경해야 함.(가끔 이 부분에서 에러 발생)
   * 'chrome driver download'로 검색 후, 사이트에 접근 후, 다운로드 가능(window, linux, mac 버전 있음)

In [2]:
from selenium import webdriver
from bs4 import BeautifulSoup

driver = webdriver.Chrome('./chromedriver_91')

url = 'https://www.bloomberg.com/graphics/covid-vaccine-tracker-global-distribution/'
driver.get(url)

In [3]:
import time
time.sleep(3)  # 홈페이지 로딩 시간 3초

### 전체 데이터 보기
 * 나라가 여러나라가 있어, 더 보기 버튼을 2번 정도 눌러준다.

In [4]:
# //*[@id="dvz-table-global-vaccination"]/div[2]/div[2]/button
# //*[@id="dvz-table-global-vaccination"]/div[2]/div[2]/button
xpath = '//*[@id="dvz-table-global-vaccination"]/div[2]/div[2]/button'
sel_more1 = driver.find_element_by_xpath(xpath)
sel_more1.click()
time.sleep(1)

In [5]:
# //*[@id="dvz-table-usa-vaccination"]/div[2]/div[2]/button
xpath = '//*[@id="dvz-table-global-vaccination"]/div[2]/div[2]/button'
sel_more2 = driver.find_element_by_xpath(xpath)
sel_more2.click()

### TABLE 선택 후, 데이터 가져오기

#### Countries and regions 
 * //*[@id="dvz-table-global-vaccination"]/div[2]/div[1]/table/tbody/tr[1]/td[1]
 * ..
 * //*[@id="dvz-table-global-vaccination"]/div[2]/div[1]/table/tbody/tr[3]/td[1]
 
#### Doses administered
 * //*[@id="dvz-table-global-vaccination"]/div[2]/div[1]/table/tbody/tr[1]/td[2]
 
#### Enough for % of people
 * //*[@id="dvz-table-global-vaccination"]/div[2]/div[1]/table/tbody/tr[1]/td[3]
 
#### given 1+ dose
 * //*[@id="dvz-table-global-vaccination"]/div[2]/div[1]/table/tbody/tr[1]/td[4]
 
#### fully vaccinated
 * //*[@id="dvz-table-global-vaccination"]/div[2]/div[1]/table/tbody/tr[1]/td[5]
 
#### Daily rate of doses administered
 * //*[@id="dvz-table-global-vaccination"]/div[2]/div[1]/table/tbody/tr[1]/td[6]
 

### 전체 데이터 가져오기

In [6]:
all_data = []

for i in range(1, 7, 1):
    data_col = []
    xpath = '//*[@id="dvz-table-global-vaccination"]/div[2]/div[1]/table/tbody/tr/td[%s]' % str(i) 
    sel_data = driver.find_elements_by_xpath(xpath)
    
    for dat in sel_data:
        #print(dat)
        data_col.append(dat.text)
    print(data_col)
    all_data.append(data_col)

['Global Total', 'Mainland China', 'India', 'EU', 'U.S.', 'Brazil', 'Indonesia', 'Japan', 'Mexico', 'Russia', 'Pakistan', 'Germany', 'Turkey', 'U.K.', '', '', '', '', '', 'Vietnam', 'France', 'Iran', 'Italy', 'Bangladesh', 'Thailand', 'South Korea', 'Philippines', 'Spain', 'Argentina', 'Canada', '', '', '', '', '', '', '', '', '', '', '', '', '', 'Colombia', 'Malaysia', 'Morocco', 'Saudi Arabia', 'Poland', 'Peru', 'Chile', 'Australia', 'Egypt', 'Uzbekistan', 'Taiwan', 'Sri Lanka', 'Cambodia', 'Cuba', 'Myanmar', 'South Africa', 'Ecuador', 'Netherlands', 'Ukraine', 'Venezuela', 'UAE', 'Belgium', 'Nepal', 'Kazakhstan', 'Portugal', 'Israel', 'Sweden', 'Romania', 'Greece', 'Dominican Republic', 'Hungary', 'Austria', 'Czech Republic', 'Algeria', 'Iraq', 'Switzerland', '', 'Tunisia', 'Azerbaijan', 'Singapore', 'Guatemala', 'Mozambique', 'Denmark', 'Nigeria', 'Hong Kong', 'El Salvador', 'Angola', 'Rwanda', 'Norway', 'Finland', 'Bolivia', 'Ireland', 'Ethiopia', 'Honduras', 'Jordan', 'Serbia', '

['–', '76.9', '31.5', '68.9', '59.1', '61.1', '34.9', '76.8', '50.7', '37.8', '24.3', '68.4', '60.5', '69.3', '', '', '', '', '', '46.8', '79.1', '54.4', '76.7', '21.4', '57.7', '78.8', '40.9', '80.8', '64.7', '76.9', '', '', '', '', '', '', '', '', '', '', '', '', '', '47.8', '76.8', '68.1', '65.6', '53.6', '52.2', '83.9', '73.1', '14.1', '17.7', '52.9', '63.0', '80.5', '80.4', '19.8', '24.3', '62.8', '73.5', '25.9', '33.3', '82.2', '75.7', '28.1', '43.1', '86.9', '66.6', '68.0', '37.7', '61.2', '65.8', '59.6', '66.4', '59.4', '12.2', '11.4', '66.4', '', '43.0', '45.6', '87.9', '22.5', '11.1', '76.6', '1.7', '60.1', '62.5', '9.2', '24.2', '71.3', '75.7', '34.2', '76.5', '1.5', '40.2', '37.8', '44.5', '54.8', '72.5', '62.0', '5.5', '75.4', '18.7', '33.6', '27.0', '68.2', '67.4', '36.9', '43.3', '26.0', '43.0', '9.3', '65.2', '79.4', '2.3', '47.2', '26.9', '4.5', '63.8', '2.8', '25.2', '78.0', '1.3', '19.4', '55.3', '6.6', '9.6', '61.2', '26.6', '33.2', '13.8', '5.7', '72.2', '16.6', '3

In [7]:
for i in range(len(all_data)):
    print(len( all_data[i]) )

226
226
226
226
226
226


In [8]:
import pandas as pd

### 데이터 확인 
 * Countries and regions : 나라 및 지역 / country 컬럼
 * Doses administered : 접종 수 / Doses_administered 컬럼
 * Enough for % of people : 접종률 / percent_of_people 컬럼
 * given 1+ dose : 1차 접종 / 1_percent 컬럼
 * fully vaccinated : 2차 접종 / 2_percent 컬럼
 * Daily_rate_of_doses_administered : 일일 투여 용량 / Daily_rate_of_doses 컬럼

In [9]:
pd.set_option("display.max_rows", 40)

dat_dict = {'국가':all_data[0],
            '백신접종수':all_data[1],
            '백신보유율(인구)':all_data[2],
            '1차접종':all_data[3],
            '2차접종':all_data[4],
            '일별접종수':all_data[5]
           }

dat_df = pd.DataFrame(dat_dict)
dat_df

,국가,백신접종수,백신보유율(인구),1차접종,2차접종,일별접종수
0,Global Total,"7,901,419,499",–,–,–,–
1,Mainland China,"2,472,847,000",176.6,87.5,76.9,–
2,India,"1,222,098,443",89.4,57.1,31.5,–
3,EU,"634,449,654",142.8,70.9,68.9,8.1
4,U.S.,"454,447,737",136.9,69.7,59.1,11.3
...,...,...,...,...,...,...
221,Nauru,"14,863",114.3,58.6,55.8,–
222,St. Helena,"7,892",131.5,72.7,58.9,–
223,Falkland Islands,"4,407",146.9,87.7,59.2,–
224,Montserrat,"2,949",59.0,30.6,28.5,–


In [10]:
dat_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 226 entries, 0 to 225
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   국가         226 non-null    object
 1   백신접종수      226 non-null    object
 2   백신보유율(인구)  226 non-null    object
 3   1차접종       226 non-null    object
 4   2차접종       226 non-null    object
 5   일별접종수      226 non-null    object
dtypes: object(6)
memory usage: 10.7+ KB


### 데이터 전처리
 * 데이터가 없거나 제대로 얻어지지 못한 부분. 그리고 추가 컬럼 등을 생성
 * [].str.len() : 데이터의 길이

In [11]:
### 공백행을 삭제
dat_df['국가'].str.len()

0      12
1      14
2       5
3       2
4       4
       ..
221     5
222    10
223    16
224    10
225     7
Name: 국가, Length: 226, dtype: int64

In [12]:
dat_df['국가'].str.len().unique()

array([12, 14,  5,  2,  4,  6,  9,  8,  7,  0, 10, 11,  3, 18, 15, 22, 19,
       21, 17, 16, 24, 13, 30], dtype=int64)

### 한나라의 중복 행의 존재로 이 부분은 정보 취득 못함

In [13]:
dat_df.loc[ dat_df['국가'].str.len() < 1, : ]

,국가,백신접종수,백신보유율(인구),1차접종,2차접종,일별접종수
14,,,,,,
15,,,,,,
16,,,,,,
17,,,,,,
18,,,,,,
30,,,,,,
31,,,,,,
32,,,,,,
33,,,,,,
34,,,,,,


### 데이터가 있는 행만 가져오기

In [14]:
print(dat_df.shape)
dat_df = dat_df.loc[ dat_df['국가'].str.len() >1, : ]
print(dat_df.shape)

(226, 6)
(207, 6)


In [15]:
dat_df.head(15)

,국가,백신접종수,백신보유율(인구),1차접종,2차접종,일별접종수
0,Global Total,"7,901,419,499",–,–,–,–
1,Mainland China,"2,472,847,000",176.6,87.5,76.9,–
2,India,"1,222,098,443",89.4,57.1,31.5,–
3,EU,"634,449,654",142.8,70.9,68.9,8.1
4,U.S.,"454,447,737",136.9,69.7,59.1,11.3
5,Brazil,"297,960,460",141.8,77.3,61.1,5.8
6,Indonesia,"232,869,968",87.2,51.5,34.9,–
7,Japan,"196,576,395",155.8,79.0,76.8,–
8,Mexico,"132,097,392",103.5,59.7,50.7,–
9,Russia,"125,544,767",85.6,44.9,37.8,2.8


### 인덱스가 일정하지 않아, 인덱스 값 초기화

In [16]:
dat_df = dat_df.reset_index()
dat_df.shape

(207, 7)

### 이상치 '-' 값 확인

In [17]:
dat_df.loc[dat_df['백신접종수'] == '–']

,index,국가,백신접종수,백신보유율(인구),1차접종,2차접종,일별접종수
206,225,Eritrea,–,–,–,–,–


In [18]:
dat_df.loc[dat_df['일별접종수'] == '–']

,index,국가,백신접종수,백신보유율(인구),1차접종,2차접종,일별접종수
0,0,Global Total,"7,901,419,499",–,–,–,–
1,1,Mainland China,"2,472,847,000",176.6,87.5,76.9,–
2,2,India,"1,222,098,443",89.4,57.1,31.5,–
6,6,Indonesia,"232,869,968",87.2,51.5,34.9,–
7,7,Japan,"196,576,395",155.8,79.0,76.8,–
...,...,...,...,...,...,...,...
202,221,Nauru,"14,863",114.3,58.6,55.8,–
203,222,St. Helena,"7,892",131.5,72.7,58.9,–
204,223,Falkland Islands,"4,407",146.9,87.7,59.2,–
205,224,Montserrat,"2,949",59.0,30.6,28.5,–


In [19]:
dat_df.columns
if 'index' in dat_df.columns:
    dat_df.drop( ['index'], axis=1, inplace=True  )
dat_df.columns

Index(['국가', '백신접종수', '백신보유율(인구)', '1차접종', '2차접종', '일별접종수'], dtype='object')

### 각 컬럼별 이상치 개수

In [20]:
col_all = dat_df.columns
for one in col_all:
    print("col name : ", one)
    print( dat_df.loc[dat_df[one] == '–', one].count() )
    print("\n")

col name :  국가
0


col name :  백신접종수
1


col name :  백신보유율(인구)
5


col name :  1차접종
5


col name :  2차접종
6


col name :  일별접종수
155




### 이상치 처리

### (1) '-'는 결측치(비어 있는 값)를 문자열 '-999'로 처리 
 * 문자열로 변경하는 이유 : 현재 컬럼이 문자열이기에
 
### (2) '<0.1'를 0.05로 임의의 값 고정값으로 변경

In [21]:
col_all = dat_df.columns
for one in col_all:
    print("col name : ", one)
    dat_df.loc[dat_df[one] == '–', one] = "-999"      # -은 이상치 -999로 치환
    dat_df.loc[dat_df[one] == '<0.1', one] = "0.05"   # <0.1은 0.05로 치환
    
    print("col name : {} 이상치 처리 OK".format(one), end="\n\n")

col name :  국가
col name : 국가 이상치 처리 OK

col name :  백신접종수
col name : 백신접종수 이상치 처리 OK

col name :  백신보유율(인구)
col name : 백신보유율(인구) 이상치 처리 OK

col name :  1차접종
col name : 1차접종 이상치 처리 OK

col name :  2차접종
col name : 2차접종 이상치 처리 OK

col name :  일별접종수
col name : 일별접종수 이상치 처리 OK



### 결측치 처리 후, 확인

In [22]:
col_all = dat_df.columns
for one in col_all:
    print("col name : ", one)
    print( dat_df.loc[dat_df[one] == '–', one].count() )

col name :  국가
0
col name :  백신접종수
0
col name :  백신보유율(인구)
0
col name :  1차접종
0
col name :  2차접종
0
col name :  일별접종수
0


### 수치 값의 천단위 구분자  ','을 처리

In [23]:

dat_df['백신접종수'] = dat_df['백신접종수'].str.replace(',', '')
dat_df['일별접종수'] = dat_df['일별접종수'].str.replace(',', '')


In [24]:
dat_df.head(10)

,국가,백신접종수,백신보유율(인구),1차접종,2차접종,일별접종수
0,Global Total,7901419499,-999,-999,-999,-999
1,Mainland China,2472847000,176.6,87.5,76.9,-999
2,India,1222098443,89.4,57.1,31.5,-999
3,EU,634449654,142.8,70.9,68.9,8.1
4,U.S.,454447737,136.9,69.7,59.1,11.3
5,Brazil,297960460,141.8,77.3,61.1,5.8
6,Indonesia,232869968,87.2,51.5,34.9,-999
7,Japan,196576395,155.8,79.0,76.8,-999
8,Mexico,132097392,103.5,59.7,50.7,-999
9,Russia,125544767,85.6,44.9,37.8,2.8


### 데이터 전처리 후, 확인

In [25]:
dat_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 207 entries, 0 to 206
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   국가         207 non-null    object
 1   백신접종수      207 non-null    object
 2   백신보유율(인구)  207 non-null    object
 3   1차접종       207 non-null    object
 4   2차접종       207 non-null    object
 5   일별접종수      207 non-null    object
dtypes: object(6)
memory usage: 9.8+ KB


In [26]:
dat_df.isnull().sum()

국가           0
백신접종수        0
백신보유율(인구)    0
1차접종         0
2차접종         0
일별접종수        0
dtype: int64

In [27]:
dat_df['국가'].unique()

array(['Global Total', 'Mainland China', 'India', 'EU', 'U.S.', 'Brazil',
       'Indonesia', 'Japan', 'Mexico', 'Russia', 'Pakistan', 'Germany',
       'Turkey', 'U.K.', 'Vietnam', 'France', 'Iran', 'Italy',
       'Bangladesh', 'Thailand', 'South Korea', 'Philippines', 'Spain',
       'Argentina', 'Canada', 'Colombia', 'Malaysia', 'Morocco',
       'Saudi Arabia', 'Poland', 'Peru', 'Chile', 'Australia', 'Egypt',
       'Uzbekistan', 'Taiwan', 'Sri Lanka', 'Cambodia', 'Cuba', 'Myanmar',
       'South Africa', 'Ecuador', 'Netherlands', 'Ukraine', 'Venezuela',
       'UAE', 'Belgium', 'Nepal', 'Kazakhstan', 'Portugal', 'Israel',
       'Sweden', 'Romania', 'Greece', 'Dominican Republic', 'Hungary',
       'Austria', 'Czech Republic', 'Algeria', 'Iraq', 'Switzerland',
       'Tunisia', 'Azerbaijan', 'Singapore', 'Guatemala', 'Mozambique',
       'Denmark', 'Nigeria', 'Hong Kong', 'El Salvador', 'Angola',
       'Rwanda', 'Norway', 'Finland', 'Bolivia', 'Ireland', 'Ethiopia',
       'Hond

In [28]:
dat_df.loc[dat_df['백신접종수'].isna() , : ] 

,국가,백신접종수,백신보유율(인구),1차접종,2차접종,일별접종수


In [29]:
dat_df.iloc[ 15:25, :]

,국가,백신접종수,백신보유율(인구),1차접종,2차접종,일별접종수
15,France,103994535,160.4,80.0,79.1,10.2
16,Iran,102145229,122.7,68.0,54.4,0.9
17,Italy,95412569,158.1,78.0,76.7,9.2
18,Bangladesh,93786656,56.3,34.2,21.4,-999
19,Thailand,91658056,131.6,68.0,57.7,4.6
20,South Korea,84150239,162.7,82.0,78.8,4.8
21,Philippines,78528357,73.2,41.5,40.9,-999
22,Spain,76026523,163.5,82.4,80.8,9.8
23,Argentina,67104574,149.3,81.0,64.7,3.5
24,Canada,60351981,160.8,79.9,76.9,-999


In [30]:
dat_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 207 entries, 0 to 206
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   국가         207 non-null    object
 1   백신접종수      207 non-null    object
 2   백신보유율(인구)  207 non-null    object
 3   1차접종       207 non-null    object
 4   2차접종       207 non-null    object
 5   일별접종수      207 non-null    object
dtypes: object(6)
memory usage: 9.8+ KB


### 수치형 변수 값 변환

In [31]:
dat_df_num = dat_df.loc[:, "백신접종수":]
dat_df_num.columns

Index(['백신접종수', '백신보유율(인구)', '1차접종', '2차접종', '일별접종수'], dtype='object')

In [32]:
sel_col = dat_df_num.columns
for one in sel_col:
    print("col name :", one)
    dat_df[one] = dat_df[one].astype('float32')
    

col name : 백신접종수
col name : 백신보유율(인구)
col name : 1차접종
col name : 2차접종
col name : 일별접종수


In [33]:
dat_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 207 entries, 0 to 206
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   국가         207 non-null    object 
 1   백신접종수      207 non-null    float32
 2   백신보유율(인구)  207 non-null    float32
 3   1차접종       207 non-null    float32
 4   2차접종       207 non-null    float32
 5   일별접종수      207 non-null    float32
dtypes: float32(5), object(1)
memory usage: 5.8+ KB


### 파일 만들기

In [37]:
from datetime import datetime
import os

now = datetime.now()
file_make_time = "%04d-%02d-%02d_%02d" % (now.year, now.month, now.day, now.hour)
print(now.day - 1)
now_day = now.day
now_hour = now.hour

print( file_make_time )

28
2021-11-29_01


In [38]:
print( os.getcwd() )
path_dir = os.getcwd() + "\\data\\"
path_file = path_dir + file_make_time
print( path_dir,  path_file, sep="\n" )

C:\Users\toto\Documents\Github\corona_analysis
C:\Users\toto\Documents\Github\corona_analysis\data\
C:\Users\toto\Documents\Github\corona_analysis\data\2021-11-29_01


In [39]:
dat_df.to_csv( path_file + "_vaccine_bloomberg.csv", index=False)
dat_df.to_excel( path_file + "_vaccine_bloomberg.xlsx", index=False)
os.listdir(path_dir)

['2021-08-07_corona.csv',
 '2021-08-07_corona.xlsx',
 '2021-09-19_corona.csv',
 '2021-09-19_corona.xlsx',
 '2021-09-20_corona.csv',
 '2021-09-20_corona.xlsx',
 '2021-09-28_corona.csv',
 '2021-09-28_corona.xlsx',
 '2021-09-29_corona.csv',
 '2021-09-29_corona.xlsx',
 '2021-11-28_corona.csv',
 '2021-11-28_corona.xlsx',
 '2021-11-29_01_vaccine_bloomberg.csv',
 '2021-11-29_01_vaccine_bloomberg.xlsx',
 '20210808_16datamerge.csv',
 '20210808_16datamerge.xlsx',
 '20210808_16_today_corona.csv',
 '20210808_16_today_corona.xlsx',
 '20210808_16_vaccine_bloomberg.csv',
 '20210808_16_vaccine_bloomberg.xlsx',
 '20210920_00_vaccine_bloomberg.csv',
 '20210920_00_vaccine_bloomberg.xlsx',
 '20210921_00_datamerge.csv',
 '20210921_00_datamerge.xlsx',
 '20210929_00_vaccine_bloomberg.csv',
 '20210929_00_vaccine_bloomberg.xlsx',
 '20210929_01_datamerge.csv',
 '20210929_01_datamerge.xlsx',
 '20210929_14_vaccine_bloomberg.csv',
 '20210929_14_vaccine_bloomberg.xlsx',
 '20210930_00_datamerge.csv',
 '20210930_00_d

* history 
 * 2021.09.30 version 01
* 출처를 밝히시고 위의 내용에 대해 자유롭게 사용 가능합니다.